In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests

from pandas import CategoricalDtype
from bs4 import BeautifulSoup
from datetime import date
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from scipy import stats
from prophet.diagnostics import cross_validation, performance_metrics

import sys
import os

backend_path = os.path.abspath('../backend')
sys.path.append(backend_path)
from get_metrics import get_metrics

In [ ]:
# чтение DataFrame df в файл data/df.csv
df = pd.read_csv('../data/df.csv')

# чтение DataFrame df_train в файл data/df_train.csv
df_train = pd.read_csv('../data/df_train.csv')

# чтение DataFrame df_test в файл data/df_test.csv
df_test = pd.read_csv('../data/df_test.csv')

# Baseline Prophet

In [ ]:
# обучение модели baseline prophet
bl_prophet = Prophet()
bl_prophet.fit(df_train)

In [ ]:
# смотрим совпадение фактического курса с предсказанным курсом baseline prophet
fig, ax = plt.subplots(figsize= (10, 5))

predict = bl_prophet.predict(df_test)
fig = bl_prophet.plot(predict, ax=ax)
ax.scatter(df_test.ds, df_test['y'], color = 'g')
ax.set_title('Прогноз ключевой ставки, Prophet на test')
ax.set_ylabel('Значение ставки')
ax.set_xlabel('Год')
plt.show()

In [ ]:
# смотрим тренд, годовые и сезонные признаки
fig = bl_prophet.plot_components(predict)
plt.show()

In [ ]:
# выводиим метрики для оценкии baseline prophet 
metrics = get_metrics(y_test=df_test['y'], y_pred=predict['yhat'], name='Baseline Prophet')
metrics

1. Mean Absolute Error (MAE) - Среднее абсолютное отклонение позволяет оценить среднее абсолютное отклонение прогнозов от фактических значений. Чем ниже MAE, тем лучше модель.
2. Mean Absolute Percentage Error (MAPE) - Среднее абсолютное процентное отклонение показывает средний процент ошибки прогноза от фактических значений. Чем ниже MAPE, тем лучше.
3. Mean Squared Error (MSE) - Среднеквадратичное отклонение предоставляет информацию о среднеквадратичном отклонении прогнозов от фактических значений. Чем ниже MSE, тем лучше модель.
4. Root Mean Squared Error (RMSE) - Квадратный корень из MSE помогает интерпретировать стандартное отклонение прогнозов от фактических значений.

# Tuning Prophet

## Hyperparameter tuning

In [ ]:
# период, который надо отрезать и предсказать (проверка модели)
pred_days = int(df.shape[0]*0.1)

In [ ]:
# выводиим метрики для оценкии tuning prophet 
#metrics = pd.concat([metrics, get_metrics(y_test=df_cv['y'], y_pred=df_cv['yhat'], name='CV Prophet')])
#metrics

In [ ]:
import numpy as np
import pandas as pd
import itertools

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

# Define parameter grid
param_grid = {
    'changepoint_prior_scale': [0.01, 0.1, 1, 10],
    'yearly_seasonality': [10, 20, 30],
    'weekly_seasonality': [5, 10, 15]
}

# Initialize list to store cross-validation results
results = []

# Loop over parameter combinations
for params in itertools.product(*param_grid.values()):
    # Create a new model with the current parameter combination
    m = Prophet(
        changepoint_prior_scale=params[0],
        yearly_seasonality=params[1],
        weekly_seasonality=params[2]
    )
    m.fit(df_train)

    # Perform cross-validation
    cv_results = cross_validation(m, horizon='365 days', period='180 days', initial='730 days')
    metrics = performance_metrics(cv_results)

    # Convert params tuple to dictionary and add to metrics
    metrics['params'] = dict(zip(['params'], [params]))

    results.append(metrics)

# Convert the results to a Pandas DataFrame
df_results = pd.DataFrame(results)

# Find the best parameters
best_params = df_results[df_results['mape'].idxmin()]

print(best_params)